# Launching LCModel

## Prepare environment

In [ ]:
!pip install vip-client


In [ ]:
from vip_client import VipSession
from pathlib import *
from os.path import exists, basename
from os import listdir, getcwd

# disable local session backup
VipSession._BACKUP_LOCATION = None

## Set up VIP execution

### First, copy paste you VIP API key from the VIP portal (in "My Account")

In [ ]:
VipSession.init(api_key="TOCHANGE");

In [ ]:
# Instantiate a Session with a name
VipSession._BACKUP_LOCATION = None
my_session = VipSession("lcmodel")

### Specify where are your input data

In [ ]:
# upload the input dataset, by default it is a "dataset" directory but you can change that
dataset = Path("dataset")

if not listdir(dataset):
    raise RuntimeError(f"[{dataset}] directory must not be empty") 
my_session.upload_inputs(dataset)

### Prepare the execution inputs

In [ ]:
# get all the signal files from the dataset
signal_files = [ dataset / signal_file for signal_file in listdir(dataset) if signal_file.lower().endswith(".raw") ]

# define the other setting from vip hosted files
input_settings = { 
    "signal_file" : signal_files,
    "zipped_folder": "/vip/MR Spectroscopy (group)/INPUT/basis.zip",
    "makebasis_file" : "/vip/MR Spectroscopy (group)/INPUT/makeBasis_3T_VIP.in",
    "control_file" : "/vip/MR Spectroscopy (group)/INPUT/fit_3T_VIP.control"
}
input_settings

## The execution is ready to be launched
### You will have to wait a few minutes for the execution to be over

In [ ]:
# launch the execution and wait for it to finish
my_session.launch_pipeline(pipeline_id = "LCModel/0.2", input_settings = input_settings)
my_session.monitor_workflows()

### Now you are ready to download the results

In [ ]:
# download the output and clean VIP data
my_session.download_outputs()
my_session.finish()

# Now you can view the pdf result(s)


In [ ]:
# This block get the pdf result local paths from the vip client info
execution_outputs = next(iter(my_session.workflows.values()))["outputs"]
#find pdf results
pdf_results = [ output for output in execution_outputs if output["path"].endswith(".pdf") ]
# get local path
prefix_length = len(my_session.vip_output_dir)+1
pdf_paths = [ Path(my_session.local_output_dir) / pdf_result["path"][prefix_length:] for pdf_result in pdf_results ]
from itertools import cycle
pdf_path_iter = cycle(pdf_paths)

# This should open a pdf viewer

## if you have several pdf results, it will display a different pdf each time you execute it

In [ ]:
pdf_path = next(pdf_path_iter)
print(f"displaying {pdf_path}") 
import IPython
IPython.display.IFrame(pdf_path, width=1000, height=600)

## This method allow to show the pdf(s) in separate tab(s)
## It should only work in JupyterLab and may fail in other python environments

Please change the prefix to the relative path of your notebook inside your jupyterlab root directory (let "/" if it is at the root).

Then execute the cell and click on the buttons below. Each button should open a jupyterlab tab with a pdf.

In [ ]:
prefix = "/"

from IPython.display import Markdown as md
md_elems = [ "<button data-commandLinker-command=\"docmanager:open\" "
            "data-commandLinker-args='{\"path\":\"" + prefix + str(pdf_path) + "\"}'  "
            "href=\"#\">Open " + basename(pdf_path) + "</button>" for pdf_path in pdf_paths ]
md("\n\n".join(md_elems))